In [102]:
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
from tqdm import tqdm
import PIL
from PIL import Image
from PIL import Image, ImageDraw, ImageOps
import os 

In [103]:
model = tf.keras.models.load_model('./saved_model',compile=False)
image_size = 75

In [104]:
# test = pd.read_csv('./test_data/test.txt', header = None, sep = '\t', names = ["image", "label"])

# for img_name in tqdm(test['image']):
#     path = './test_data/test_images/'+str(img_name)
#     with PIL.Image.open(path) as img:
#         width, height = img.size
#         left_half = (0, 0, width//4,height)
#         img = img.crop(left_half)
#         img = img.resize((299,299))
#         img.save(f'./{"resize_test"}/{img_name}') 

In [105]:
import cv2
import os.path
import pandas as pd
import PIL
import urllib.request
from tqdm import tqdm

# Define the URL and local filename of the Haar Cascade classifier file
url = 'https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_frontalface_default.xml'
local_file = 'haar_cascade/haarcascade_frontalface_default.xml'

# Download the Haar Cascade classifier file if it doesn't exist
if not os.path.exists(local_file):
    urllib.request.urlretrieve(url, local_file)

# Load the face detection classifier
face_cascade = cv2.CascadeClassifier(local_file)

# Load the test data
test = pd.read_csv('./test_data/test.txt', header=None, sep='\t', names=['image', 'label'])

# Define a function to detect and crop faces from an image
def detect_and_crop_faces(image_path):
    with PIL.Image.open(image_path) as img:
        # Upscale the image
        img = img.resize((img.size[0]*4, img.size[1]*4), resample=PIL.Image.BICUBIC)
        
        # Crop the left half of the image
        width, height = img.size
        left_half = (0, 0, width//4, height)
        img = img.crop(left_half)
        
        # Convert the image to grayscale
        gray = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2GRAY)
        
        # Detect faces in the image
        faces = face_cascade.detectMultiScale(gray, scaleFactor=1.2, minNeighbors=3, minSize=(50, 50))
        
        # Get the face closest to the center of the image
        center_x, center_y = img.size[0] // 2, img.size[1] // 2
        closest_face = None
        closest_distance = float('inf')
        for (x, y, w, h) in faces:
            distance = ((x + w // 2) - center_x) ** 2 + ((y + h // 2) - center_y) ** 2
            if distance < closest_distance:
                closest_face = (x, y, w, h)
                closest_distance = distance
        
        # Crop and resize the closest face
        if closest_face is not None:
            (x, y, w, h) = closest_face
            # Crop the face
            face_img = img.crop((x, y, x+w, y+h))
            
            # Resize the face to a specific size
            resized_face = face_img.resize((75, 75))
            
            # Save the cropped and resized face
            return resized_face
        else:
            # Return None if no faces are detected
            return None        
        
# Loop over each image in the test set and detect and crop faces
for img_name in tqdm(test['image']):
    # Load the image
    path = './test_data/test_images/' + str(img_name)
    
    # Detect and crop faces from the image
    cropped_faces = detect_and_crop_faces(path)
    
    # Save the cropped faces
    if cropped_faces is not None:
        cropped_faces.save(f'./{"resize_test"}/{img_name}')
    else:
        input_image_path = os.path.join(path)
        with Image.open(input_image_path) as input_image:
            width, height = input_image.size
            left_half = (0, 0, width//5, height)
            cropped_image = input_image.crop(left_half)
            cropped_image.save(f'./{"resize_test"}/{img_name}')

100%|█████████████████████████████████████████████████████| 98/98 [00:01<00:00, 49.63it/s]


In [106]:
# test['image'] = test['image'].str.replace('.jpg', '.png')

datagen = keras.preprocessing.image.ImageDataGenerator(rescale=1/255.0,
                                                        preprocessing_function=None,
                                                        data_format=None,
                                                    )
test_data = datagen.flow_from_dataframe(
    test,
    directory = './resize_test',
    x_col="image",
    y_col= None,
    color_mode="rgb",
    target_size = (image_size,image_size),
    classes=None,
    class_mode=None,
    batch_size=32,
    shuffle=False,
    seed=40,
)

preds = model.predict(test_data)
# print(preds)
preds = preds.tolist()

indices = []
for pred in preds:
#     print(pred)
    max_pred= 0.0
    temp = []
    for category in pred:
        if category>max_pred:
#             temp.append(pred.index(category))
#             break
            max_pred = category
    temp.append(pred.index(max_pred))
    if temp!=[]:
        indices.append(temp)
    else:
        temp.append(np.argmax(pred))
        indices.append(temp)

# print(indices)

Found 98 validated image filenames.


2023-05-14 14:51:13.638053: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]


4/4 [==============================] - 3s 224ms/step


In [107]:
train = pd.read_csv('train.csv', header=None, names=["image", "labels"])
train_data = datagen.flow_from_dataframe(
    train,
    directory='./train_dataset',
    x_col="image",
    y_col= 'labels',
    color_mode="rgb",
    target_size = (image_size,image_size),
    class_mode="categorical",
    batch_size=4,
    shuffle=False,
    seed=40,
)
labels = (train_data.class_indices)
labels = dict((v,k) for k,v in labels.items())
print(labels)

testlabels = []


for image in indices:
    temp = []
    for i in image:
        temp.append(str(labels[i]))
    testlabels.append(' '.join(temp))

print(testlabels)

Found 64 validated image filenames belonging to 64 classes.
{0: 'Ahri', 1: 'Akali', 2: 'Alistar', 3: 'Amumu', 4: 'Annie', 5: 'Ashe', 6: 'AurelionSol', 7: 'Blitzcrank', 8: 'Braum', 9: 'Camille', 10: 'Corki', 11: 'Darius', 12: 'Diana', 13: 'DrMundo', 14: 'Draven', 15: 'Evelynn', 16: 'Ezreal', 17: 'Fiora', 18: 'Fizz', 19: 'Galio', 20: 'Garen', 21: 'Gragas', 22: 'Graves', 23: 'Janna', 24: 'JarvanIV', 25: 'Jax', 26: 'Jhin', 27: 'Jinx', 28: 'Kaisa', 29: 'Katarina', 30: 'Kennen', 31: 'Khazix', 32: 'LeeSin', 33: 'Leona', 34: 'Lulu', 35: 'Lux', 36: 'Malphite', 37: 'MasterYi', 38: 'MissFortune', 39: 'MonkeyKing', 40: 'Nami', 41: 'Nasus', 42: 'Olaf', 43: 'Orianna', 44: 'Pantheon', 45: 'Rakan', 46: 'Rammus', 47: 'Rengar', 48: 'Seraphine', 49: 'Shyvana', 50: 'Singed', 51: 'Sona', 52: 'Soraka', 53: 'Teemo', 54: 'Tristana', 55: 'Tryndamere', 56: 'TwistedFate', 57: 'Varus', 58: 'Vayne', 59: 'Vi', 60: 'XinZhao', 61: 'Yasuo', 62: 'Zed', 63: 'Ziggs'}
['Tryndamere', 'Galio', 'Amumu', 'Akali', 'MonkeyKing'

In [108]:
test['predict'] = testlabels
df = pd.DataFrame(test)
df.to_csv('test.csv', index=False)

In [109]:
test

,image,label,predict
0,Jinx_630613568339321_round1_Dr.-Mundo_06-04-20...,Jinx,Tryndamere
1,Darius_1115082439004174_round1_Darius_05-19-20...,Darius,Galio
2,Jinx_v1033497503_round5_Sona_05-25-2021.mp4_85...,Jinx,Amumu
3,Miss_232154245077306_round1_Lee-Sin_06-14-2021...,Miss_Fortune,Akali
4,Jinx_182388960412791_round6_Jinx_05-04-2021.mp...,Jinx,MonkeyKing
...,...,...,...
93,Ahri_278220660753197_round6_Ahri_06-02-2021.mp...,Ahri,Orianna
94,Annie_2996176217280056_round1_Akali_06-11-2021...,Annie,Amumu
95,Dr_1140587809775566_round3_Dr.-Mundo_06-08-202...,Dr._Mundo,DrMundo
96,Annie_4448886761788772_round5_Varus_05-04-2021...,Annie,Amumu
